In [19]:
import os
import json
import cpi

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import pymongo
path = "../Dataset/BoxOfficeMojo_with_imdb_ID/BoxOfficeMojo/"

In [69]:
import copy
from tqdm import tqdm

In [20]:
years = os.listdir(path)
years = sorted(years)

In [21]:
num_movies = []
for year in years:
    movie_list = os.listdir(path + year)
    if(year < "2021"):
        num_movies.append(len(movie_list)-1)

In [22]:
fig = go.Figure([go.Bar(x=years, y=num_movies)])
fig.show()

In [23]:
len(movie_list)

333

In [24]:
def mergeDict(dict1, dict2):
    ret = {}
    for key in dict1:
        ret[key] = dict1[key]
    for key in dict2:
        ret[key] = dict2[key]
    return ret

In [25]:
def load_movies(year):
    with open(path+year+"/0_summary.json", 'r') as f:
        summary = json.load(f)
    print("found {} movies in the year {}".format(len(summary), year))
    cnt = 0
    movie_list = []
    for movie in summary:
        cnt+=1
    #     print("loading {} out of {} -- ".format(cnt, len(summary)) + movie['name'])
        file_name = str(cnt) + "_" + movie['file_name']
        try: 
            with open(path + year + "/" + file_name + ".json", 'r') as f:
                movie_json = json.load(f)
                movie_json = mergeDict(movie, movie_json)
                movie_list.append(movie_json)
#                 print(movie_json)
        except:
            print("NOT FOUND!!! >> {} -- {}".format(file_name, year))
#         break
    return movie_list

In [26]:
all_movies = []
for year in years:
    all_movies += load_movies(year)

found 35 movies in the year 1977
found 55 movies in the year 1978
found 84 movies in the year 1979
found 116 movies in the year 1980
found 114 movies in the year 1981
found 132 movies in the year 1982
found 159 movies in the year 1983
found 165 movies in the year 1984
found 180 movies in the year 1985
found 208 movies in the year 1986
found 234 movies in the year 1987
found 248 movies in the year 1988
found 236 movies in the year 1989
found 224 movies in the year 1990
found 245 movies in the year 1991
NOT FOUND!!! >> 48_necessary_roughness -- 1991
found 236 movies in the year 1992
found 260 movies in the year 1993
found 257 movies in the year 1994
found 282 movies in the year 1995
found 312 movies in the year 1996
found 305 movies in the year 1997
found 341 movies in the year 1998
found 396 movies in the year 1999
found 414 movies in the year 2000
found 690 movies in the year 2001
found 1374 movies in the year 2002
found 1396 movies in the year 2003
NOT FOUND!!! >> 268_the_spanish_apar

In [27]:
len(all_movies)

51013

In [28]:
def getIMDB_Id(movie):
    pro_url = movie['General Info']['IMDbPro']
    id = pro_url.split('?')[0].split('/')[-1]
    return id

In [29]:
getIMDB_Id(all_movies[10])

'tt0076538'

In [54]:
test_movie = all_movies[11010]
test_movie

{'href': '/releasegroup/gr1021399557/?ref_=bo_ydw_table_1056',
 'name': 'Pasos',
 'file_name': 'pasos',
 'world_wide': '$38,132',
 'domestic': '-',
 'foreign': '$38,132',
 'General Info': {'Distributor': 'Alta Films',
  'Opening': '$38,132',
  'Release Date': ['Jun 17, 2005'],
  'Running Time': '1 hr 44 min',
  'Genres': ['Drama'],
  'In Release': '198 days/28 weeks',
  'Widest Release': '26 theaters',
  'IMDbPro': 'https://pro.imdb.com/title/tt0441430?ref_=mojo_rl_summary&rf=mojo_rl_summary'},
 'Weekly Income': 'N/A -- checked on 2021-04-27 06:41:22.503102'}

In [31]:
def getUSD(txt):
    # converter = {
    #     '$'  : 1, 'USD': 1, 'EUR': 1.22, 'AUD': 0.78,
    #     'GBP': 1.41, 'NOK': 0.12, 'DKK': 0.16, 'JPY': 0.0092,
    #     'CAD': 0.83, 'INR': 0.012, 'SEK': 0.12, 'DEM': 0.62,
    #     'FRF': 0.185, 'RUR': 0.014, 'CNY': 0.16, 'BEF': 1/33.5,
    #     'MXN': 0.050, 'CZK': 0.048, 'ZAR': 0.071, 'HKD': 0.13,
    #     'BRL': 0.19, 'IDR': 0.000070, 'ITL': 0.000626628, 'ARS': 0.011,
    #     'UAH': 0.036, 'NZD': 0.73, 'TRL': 0.12, 'FIM': 0.204585,
    #     'HUF': 0.0034, 'DOP': 0.018, 'KRW': 0.00089, 'ESP': 1.22, 
    #     'EGP': 0.064, 'PLN': 0.27, 'SGD': 0.75, 'IQD': 0.00068, 
    #     'IEP': 0.647361, 'CHF': 1.11, 'ISK': 0.0081, 'TWD': 0.036,
    #     'THB': 0.032, 'MYR': 0.24, 'PYG': 0.00015, 'NGN': 0.0026, 
    #     'ILS': 0.30, 'IRR': 0.000024, 'NLG': 0.039, 'BDT': 0.012, 
    #     'AMD': 0.0019, 'MVR': 0.065, 'SAR': 0.27, 'RON': 0.25,
    #     'ATS': 0.0883949, 'PHP': 0.021, 'ROL': 0.25
    # }
    converter = {
        '$'  : 1, 'USD': 1
    }
    txt = txt.replace(",","")
    mul = 1
    for key in converter:
        if key in txt:
            txt = txt.replace(key, "")
            mul = converter[key]
            break
    money = int(txt)
    return money

In [32]:
def adjustInflation(amount, year):
    try:
        year = int(year)
        return int(cpi.inflate(amount, year))
    except:
        return amount

In [43]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = []
for year in range(1967, 2021):
    years.append(str(year))

In [44]:
def processReleaseDate(date):
    if(type(date) == list):
        date = date[0]
    month, day, year = date.replace(',','').split(" ")
    if(month not in months):
        month = None
    if(year not in year):
        year = None
    return month, day, year

In [57]:
def cleanMovie(movie, print_log = False):
    movie = copy.deepcopy(movie)
    movie['imdb_id'] = getIMDB_Id(movie)

    # date
    try:
        month, day, year = processReleaseDate(movie['General Info']['Release Date'])
        if(month == None or year == None):
            if(print_log == True):
                print("{} Problem loading month {} or year {} info".format(movie['imdb_id'],month, year))
            return None
        movie['cleaned_month'] = month
        movie['cleaned_year'] = year
    except:
        if(print_log == True):
            print(movie['imdb_id'], "Problem parsing month {} or year {}".format(month, year))
        return None

    # revenue
    if('world_wide' not in movie):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <world_wide> attribute")
        return None
    revenue = movie['world_wide']
    try:
        revenue = getUSD(revenue)
    except:
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <world_wide> attribute in a convertable format")
        return None
    movie['cleaned_Revenue'] = adjustInflation(revenue, movie['cleaned_year'])

    #'Widest Release'
    if('Widest Release' not in movie['General Info']):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Widest Release> attribute")
        return None
    try:
        release = movie['General Info']['Widest Release']
        release = int(release.split()[0])
    except:
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Widest Release> attribute in a convertable format")
        return None
    movie['cleaned_release_theaters'] = release
    return movie

In [58]:
cleanMovie(test_movie, print_log=True)

{'href': '/releasegroup/gr1021399557/?ref_=bo_ydw_table_1056',
 'name': 'Pasos',
 'file_name': 'pasos',
 'world_wide': '$38,132',
 'domestic': '-',
 'foreign': '$38,132',
 'General Info': {'Distributor': 'Alta Films',
  'Opening': '$38,132',
  'Release Date': ['Jun 17, 2005'],
  'Running Time': '1 hr 44 min',
  'Genres': ['Drama'],
  'In Release': '198 days/28 weeks',
  'Widest Release': '26 theaters',
  'IMDbPro': 'https://pro.imdb.com/title/tt0441430?ref_=mojo_rl_summary&rf=mojo_rl_summary'},
 'Weekly Income': 'N/A -- checked on 2021-04-27 06:41:22.503102',
 'imdb_id': 'tt0441430',
 'cleaned_month': 'Jun',
 'cleaned_year': '2005',
 'cleaned_Revenue': 50532,
 'cleaned_release_theaters': 26}

In [78]:
clean_failed = 0
clean_success = 0
limit = len(all_movies)
step_size = 1000
cnt = 0

movie_list = []
for movie in all_movies:
    try:
        cleaned_movie = cleanMovie(movie)
        if(cleaned_movie == None):
            clean_failed += 1
        else:
            clean_success += 1
            movie_list.append(cleaned_movie)
    except:
        print(movie['imdb_id'])
        break
    cnt += 1
    if(cnt % step_size == 0):
        print("processed {} out of {} --- success {} , fail {}".format(cnt, limit, clean_success, clean_failed))

print('processed {} -- failed {}'.format(clean_success, clean_failed))

processed 1000 out of 51013 --- success 420 , fail 580
processed 2000 out of 51013 --- success 880 , fail 1120
processed 3000 out of 51013 --- success 1333 , fail 1667
processed 4000 out of 51013 --- success 1749 , fail 2251
processed 5000 out of 51013 --- success 2283 , fail 2717
processed 6000 out of 51013 --- success 3026 , fail 2974
processed 7000 out of 51013 --- success 3840 , fail 3160
processed 8000 out of 51013 --- success 4610 , fail 3390
processed 9000 out of 51013 --- success 5392 , fail 3608
processed 10000 out of 51013 --- success 6263 , fail 3737
processed 11000 out of 51013 --- success 7049 , fail 3951
processed 12000 out of 51013 --- success 7821 , fail 4179
processed 13000 out of 51013 --- success 8689 , fail 4311
processed 14000 out of 51013 --- success 9448 , fail 4552
processed 15000 out of 51013 --- success 10312 , fail 4688
processed 16000 out of 51013 --- success 11090 , fail 4910
processed 17000 out of 51013 --- success 11999 , fail 5001
processed 18000 out of 

In [72]:
len(movie_list)

40434

In [73]:
import pymongo

In [81]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["boxOfficeMojo"]
bmojo = db['movies']

In [80]:
# bmojo.drop()

In [83]:
for i in tqdm(range(0, len(movie_list))):
    bmojo.insert_one(movie_list[i])

100%|██████████| 40434/40434 [00:37<00:00, 1070.98it/s]


In [84]:
bmojo.count_documents({})

40434